In [ ]:
cd ~
pwd

In [ ]:
!git clone https://github.com/minghui-liu/cold-compress.git
cd cold-compress

In [ ]:
!pip install --user -r requirements.txt --extra-index-url https://download.pytorch.org/whl/nightly/

In [ ]:
!git config --global credential.helper store

!echo "HUGGINGFACE_TOKEN=[]" > .env
!echo "OPENAI_API_KEY=[]" >> .env
!cat .env
!export $(grep -v '^#' .env | xargs -d '\n')
!huggingface-cli login --token [your token here] --add-to-git-credential

In [ ]:
!bash scripts/prepare_llama31.sh
!bash scripts/prepare_qwen2.sh

In [ ]:
pwd

In [ ]:
!bash prepare_data.sh qwen2-7b-chat synthetic

In [ ]:
######### Create RULER dataset of various length #########
from __future__ import annotations

import json
import os
from typing import Any, List, Union

def read_json_objects(file_path: Union[str, os.PathLike]) -> Union[Any, List[Any]]:
    """
    Read a JSON/JSONL file and return its contents.

    - If `file_path` ends with .json -> returns the JSON object (dict, list, etc.).
    - If `file_path` ends with .jsonl or .ndjson -> returns a list of JSON objects, one per line.
    - If the extension is ambiguous, it first tries to `json.load`; on failure it
      falls back to parsing it as JSON Lines.

    Parameters
    ----------
    file_path : str | os.PathLike
        Path to the file.

    Returns
    -------
    Any | list[Any]
        A JSON object for .json files, or a list of JSON objects for .jsonl/.ndjson.

    Raises
    ------
    FileNotFoundError
        If the file does not exist.
    json.JSONDecodeError
        If the file cannot be decoded as JSON or JSON Lines.
    """
    path_str = os.fspath(file_path)
    is_jsonl = path_str.endswith((".jsonl", ".ndjson"))

    with open(file_path, "r", encoding="utf-8") as f:
        if is_jsonl:
            return [json.loads(line) for line in f if line.strip()]

        # Try standard JSON first
        try:
            return json.load(f)
        except json.JSONDecodeError:
            # Fall back to JSON Lines parsing
            f.seek(0)
            try:
                return [json.loads(line) for line in f if line.strip()]
            except json.JSONDecodeError as e:
                # Re-raise with the original context
                raise e 

######## llama model ########
from datasets import Dataset, DatasetDict

tasks = [
    "niah_single_1",
    "niah_single_2",
    "niah_single_3",
    "niah_multikey_1",
    "niah_multikey_2",
    "niah_multikey_3",
    "niah_multivalue",
    "niah_multiquery",
    "vt",
    "cwe",
    "fwe",
    "qa_1",
    "qa_2"
]

seq_lengths = [131072, 65536, 32768, 16384, 8192]
seq_len_names = {
    131072: "131k",
    65536: "64k",
    32768: "32k",
    16384: "16k",
    8192: "8k",
}

model_name = 'llama3.1-8b-chat'
benchmark_root = '/home/ubuntu/hashevict/RULER/scripts/benchmark_root'
data_root = os.path.join(benchmark_root, model_name, 'synthetic')

my_dataset_dict = {}
for task in tasks:
    for seq_len in seq_lengths:
        # create subset
        subset_name = f"{task}_{seq_len_names[seq_len]}"
        subset_json_file = os.path.join(data_root, str(seq_len), 'data', task, 'validation.jsonl')
        subset_data = read_json_objects(subset_json_file)
        my_dataset_dict[subset_name] = Dataset.from_list(subset_data)
        
print(my_dataset_dict.keys())

repo_id = 'minghuiliu/ruler_llama'
for key in my_dataset_dict:
    my_dataset_dict[key].push_to_hub(repo_id, config_name=key, split="test")



In [ ]:

######## qwen2 model ########
model_name = 'qwen2-7b-chat'
benchmark_root = '/home/ubuntu/hashevict/RULER/scripts/benchmark_root'
data_root = os.path.join(benchmark_root, model_name, 'synthetic')

my_dataset_dict = {}
for task in tasks:
    for seq_len in seq_lengths:
        # create subset
        subset_name = f"{task}_{seq_len_names[seq_len]}"
        subset_json_file = os.path.join(data_root, str(seq_len), 'data', task, 'validation.jsonl')
        subset_data = read_json_objects(subset_json_file)
        my_dataset_dict[subset_name] = Dataset.from_list(subset_data)
        
print(my_dataset_dict.keys())

repo_id = 'minghuiliu/ruler_qwen'
for key in my_dataset_dict:
    my_dataset_dict[key].push_to_hub(repo_id, config_name=key, split="test")